In [1]:
import warnings
warnings.simplefilter("ignore")
from utils.helpers import *
from Evaluator import *
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
import ipywidgets as widets
%matplotlib inline
pred_model= PredictionModel(False,)

[nltk_data] Downloading package stopwords to /home/jojo89/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# UI Components
lbl_title = widets.Label(value='Enter Title: ')
title_box = widets.Text()

lbl_content = widets.Label(value='Enter Content: ',)
content_box = widets.Textarea(layout=widets.Layout(width='460px',height='350px'))
btn_send = widets.Button(description='Submit')
btn_send_out = widets.Output()
ttl_widget= widets.Label(value='Fake News Detector',layout=widets.Layout(margin='0px 60px',font_weight='large'))
ttl_widget.style.font_weight='large'

# Add two progressbars to show the probability scores
lbl_final_l= widets.Label(value='Label: ')
lbl_final_label= widets.Label(value='')
real_float= widets.FloatProgress(min=0.0,max=1.0,)
real_float.style.bar_color='green'

fake_float= widets.FloatProgress(min=0.0,max=1.0,)
fake_float.style.bar_color='red'
results_status=widets.VBox([widets.HBox([lbl_final_l,lbl_final_label])
    ,
    widets.HBox([widets.Label(value='Real  '),real_float]),
widets.HBox([widets.Label(value='Fake  '),fake_float])])
results_status.layout.display='hidden'

empty_lbl = widets.Label(value='Please provide the Title and Text of the article')
empty_lbl.layout.display='hidden'

In [ ]:
def getJointLabels(predictions,content_type='news content'):
    labels = []
    probs=[]
    if type(predictions)!=list:
        predictions = [predictions]
    for pred in predictions:
        if pred is None:
            continue
        if type(pred) == list:
            for p in pred:
                labels.append(p['labels'])
                if type(p['probability_scores'])!=str:
                    probs.append(p['probability_scores'])
        else:
            labels.append(pred['labels'])
            if type(pred['probability_scores'])!=str:
                    probs.append(pred['probability_scores'])
    joint_label = Counter(labels).most_common(1)[0][0]
    
    #print(f"The given {content_type} is: {joint_label}")
    try:
        probs= np.array(probs)
        probs=np.mean(probs,0)
        #print(f'Chances of being fake is: {probs[0][0]}')
    except:
        pass
    return joint_label,probs
def detecting_fake_news(title,text):  
    content_prediction=title_prediction=text_prediction = None
    if title.strip()!='':
        title_prediction= pred_model.predict(title,text_type='title',return_all_models=False)
        
    # Make Prediction Based on only the text
    if text.strip()!='':
        text_prediction= pred_model.predict(text,text_type='text',return_all_models=False)
        
    #Make prediction based on the text and title
    if title.strip()!='' and text.strip()!='':
        content_prediction = pred_model.predict(title+' '+text,text_type='content',return_all_models=False)
        
    label,prob=getJointLabels([content_prediction,
                                   title_prediction,text_prediction],
                                  content_type='news content')
    
    return label,prob

# Handle the onclick operation
def generateLabel(b):
    
    with btn_send_out:
        results_status.layout.display='hidden'
        empty_lbl.layout.display ='hidden'
        real_float.value=0.0
        fake_float.value=0.0
        if content_box.value.strip()!='':
            label,prob= detecting_fake_news(title_box.value,
                                            content_box.value)
            real_float.value=prob[0][1]
            fake_float.value= prob[0][0]
            lbl_final_label.value= label.upper()
            results_status.layout.display='visible'
        else:
            lbl_final_label.value= ''
            results_status.layout.display='hidden'
            empty_lbl.layout.display='visible'
        
        
btn_send.on_click(generateLabel)
btn_send.layout= widets.Layout(margin='10px 180px',)

In [ ]:
# Show the UI for interaction
widets.VBox([ttl_widget,
             lbl_title,title_box,lbl_content,
             content_box,btn_send,btn_send_out,results_status],
            
            layout=widets.Layout(margin='0px 190px',))

In [ ]:
#widets.VBox([widets.HBox([lbl_title,title_box]),widets.HBox([lbl_content,content_box]),btn_send])
#display(btn_send,btn_send_out)

In [ ]:
#margin: 0px 60px;font-size:large;font-weight:bolder"